<a href="https://www.kaggle.com/code/samithsachidanandan/simple-neural-network-with-pytorch?scriptVersionId=298093364" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils .data import DataLoader, TensorDataset

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 

# Load and Prepare Data

In [2]:
X, y =load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch Tensors

In [3]:
X_train_scaled_tensor = torch.from_numpy(X_train_scaled).float()
X_test_scaled_tensor = torch.from_numpy(X_test_scaled).float()
y_train_tensor = torch.from_numpy(y_train).float().unsqueeze(1)
y_test_tensor = torch.from_numpy(y_test).float().unsqueeze(1)

# Create Dataset and DataLoader

In [4]:
train_dataset = TensorDataset(X_train_scaled_tensor,y_train_tensor )

In [5]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Neural Network Architecture

In [6]:
class BCNet(nn.Module):
    
    def __init__(self):
        super(BCNet,self).__init__()
        self.fc1 = nn.Linear(30, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.sigmoid(self.fc3(x))
        return x


# Initialize Model

In [7]:
model = BCNet()

# Setup Loss Function and Optimizer

In [8]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

# Training Loop

In [9]:
epochs = 50 

for epoch in range(epochs):
    model.train()
    running_loss = 0.0 

    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        
        preds = model(x_batch)
        loss = criterion(preds, y_batch)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch+1}: Loss was {running_loss / len(train_loader)}')

Epoch 1: Loss was 0.6530991276105245
Epoch 2: Loss was 0.5418427944183349
Epoch 3: Loss was 0.4020681103070577
Epoch 4: Loss was 0.27755389412244164
Epoch 5: Loss was 0.17738686601320902
Epoch 6: Loss was 0.13534238984187444
Epoch 7: Loss was 0.12062061900893847
Epoch 8: Loss was 0.09719100104024013
Epoch 9: Loss was 0.09198675180474918
Epoch 10: Loss was 0.07853146642446518
Epoch 11: Loss was 0.07657907990117868
Epoch 12: Loss was 0.06430309290687243
Epoch 13: Loss was 0.06855583234379689
Epoch 14: Loss was 0.07434393713871638
Epoch 15: Loss was 0.06862513416757186
Epoch 16: Loss was 0.06236007828265429
Epoch 17: Loss was 0.06180583816021681
Epoch 18: Loss was 0.04996816525235772
Epoch 19: Loss was 0.05716599430888891
Epoch 20: Loss was 0.055418373954792814
Epoch 21: Loss was 0.04772063239167134
Epoch 22: Loss was 0.04534283233806491
Epoch 23: Loss was 0.04633380918142696
Epoch 24: Loss was 0.04671735074371099
Epoch 25: Loss was 0.04745339665872355
Epoch 26: Loss was 0.045273538449934

# Evaluate Model on Test Set

In [10]:
with torch.no_grad():
    model.eval()

    preds = model(X_test_scaled_tensor)
    loss = criterion(preds, y_test_tensor).item()

    accuracy = ((preds >= 0.5) == y_test_tensor).float().mean().item()

# Display Accuracy

In [11]:
accuracy

0.9824561476707458